# Advanced output handling


For grasping the basic idea of output handlers, check out this introduction: [validating agent output](validating_agent_output.html).

The purpose of this example is to demonstrate two concepts:

- Obtaining structured output from the agent
- Validating the output against the prompt that was given to the agent

This should give a good understanding of output handler semantics and prepare you to build your own ones.

In [1]:
from dotenv import load_dotenv
load_dotenv()

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.tasks import SimpleTask
from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools import PythonREPLTool
from motleycrew.tools import LLMTool
from motleycrew.agents import MotleyOutputHandler

In the [validating agent output](validating_agent_output.html) example, we used a simple tool to receive and check the output.

Here we want to have access to the agent input, so we subclass a special `MotleyOutputHandler` class and implement its `handle_output` method. Inside it, we'll call an LLM critic that will assess the agent's output.

In [2]:
class CoderOutputHandler(MotleyOutputHandler):
    def handle_output(self, code: str, comment: str):
        agent_prompt = self.agent_input["prompt"]
        critic = LLMTool(
            name="critic",
            description="improve the agent output",
            prompt=(
                "An AI agent was given this prompt: \n{agent_prompt}\n\n"
                "Here's the code it generated: \n```\n{code}\n```\n\n"
                "And the comment it gave: \n{comment}\n\n"
                "The code and the comment must be sensible and easily understandable. "
                "Give constructive advice on how to improve them unless they are already perfect. "
                "In this case, just return a single \"OK\" without quotes."
            )
        )

        critic_message = critic.invoke({"agent_prompt": agent_prompt, "code": code, "comment": comment}).content
        print("Critic response: ", critic_message)

        if critic_message.strip().lower().startswith("ok"):
            return code, comment
        raise InvalidOutput(critic_message)  # This will be returned to the agent to make it improve its output

In case the critic is not satisfied, the `InvalidOutput` exception will be returned to the agent, as if a regular tool was called. If everything is OK, we just return the agent's output.

The MotleyOutputHandler constructor has a `max_iterations` argument. It's the maximum number of times the exception will be returned to the agent. If the agent doesn't provide a valid output after `max_iterations`, a `OutputHandlerMaxIterationsExceeded` exception will be raised. It has `last_call_args` and `last_call_kwargs` attributes that store the arguments that the agent used in the last call. Also, it stores the last exception that was raised in the `last_exception` attribute.

The output schema is inferred automatically from the `handle_output` method signature. You can also specify it directly using the `_args_schema` property (see below).

In [3]:
crew = MotleyCrew()

coder = ReActToolCallingMotleyAgent(
    name="coder",
    tools=[PythonREPLTool()],
    output_handler=CoderOutputHandler(max_iterations=3),
    verbose=True,
)

task = SimpleTask(
    crew=crew,
    name="Explain and implement the bubble sorting algorithm",
    description="Write a Python program that implements the bubble sorting algorithm "
                "and test your implementation using the REPL tool. \n"
                "Return the code and a comment explaining how it works.",
    agent=coder,
)

2024-07-19 16:55:50,737 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string
2024-07-19 16:55:50,741 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[dict[str, typing.Any]], will use JSON string


In [4]:
crew.run()
code, comment = task.output

2024-07-19 16:55:50,774 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string
2024-07-19 16:55:50,775 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[dict[str, typing.Any]], will use JSON string




> Entering new AgentExecutor chain...



Invoking: `python_repl` with `{'command': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1, arr[j]]\n    return arr\n\n# Test the bubble sort function\nsample_list = [64, 34, 25, 12, 22, 11, 90]\nsorted_list = bubble_sort(sample_list)\nprint(sorted_list)'}`
responded: Thought: The next step is to write the Python code for the bubble sort algorithm. After writing the code, I will test it using the REPL tool to ensure it works correctly. Finally, I will return the code along with an explanation of how it works.

Let's start by writing the bubble sort algorithm and testing it with a sample list.



TypeError('list indices must be integers or slices, not tuple')
Invoking: `python_repl` with `{'command': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], 

2024-07-19 16:56:50,731 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string


Critic response:  OK
('def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        swapped = False\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n                swapped = True\n        if not swapped:\n            break\n    return arr\n\n# Test the bubble sort function\nsample_list = [64, 34, 25, 12, 22, 11, 90]\nsorted_list = bubble_sort(sample_list)\nprint(sorted_list)', "The bubble sort algorithm works by repeatedly stepping through the list to be sorted, comparing each pair of adjacent items and swapping them if they are in the wrong order. This process is repeated until the list is sorted. The algorithm gets its name because smaller elements 'bubble' to the top of the list. The outer loop runs 'n' times, where 'n' is the length of the list, and the inner loop runs 'n-i-1' times to avoid re-checking the already sorted elements at the end of the list. An optimization is added to exit early if

See, the initial code was optimized thanks to the critic!

In [5]:
from IPython.display import display, Markdown, Code

display(Code(code, language="python"))

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        swapped = False
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True
        if not swapped:
            break
    return arr

# Test the bubble sort function
sample_list = [64, 34, 25, 12, 22, 11, 90]
sorted_list = bubble_sort(sample_list)
print(sorted_list)

In [6]:
display(Markdown(comment))

The bubble sort algorithm works by repeatedly stepping through the list to be sorted, comparing each pair of adjacent items and swapping them if they are in the wrong order. This process is repeated until the list is sorted. The algorithm gets its name because smaller elements 'bubble' to the top of the list. The outer loop runs 'n' times, where 'n' is the length of the list, and the inner loop runs 'n-i-1' times to avoid re-checking the already sorted elements at the end of the list. An optimization is added to exit early if no swaps are made during an iteration, indicating that the list is already sorted. The time complexity of bubble sort is O(n^2) in the worst and average cases, but it can be O(n) if the list is already sorted.

In [7]:
final_result = f"{code}\n\n{comment}"

## Customizing the output handler

When subclassing `MotleyOutputHandler`, you can override the default name, description and args schema. You can also specify which exceptions will be caught and returned to the agent if raised inside the output handler.

In [8]:
from langchain_core.pydantic_v1 import BaseModel, Field

class CustomOutputHandlerSchema(BaseModel):
    code = Field(str, description="Python code demonstrating bubble sort")
    comment = Field(str, description="Comment explaining how the code works")


class MyCustomOutputHandler(MotleyOutputHandler):
    _name = "Output handler"
    _description = "Output handler. ONLY RETURN THE FINAL RESULT USING THIS TOOL!"
    _args_schema = CustomOutputHandlerSchema
    _exceptions_to_handle = (InvalidOutput, ValueError)  # ValueErrors will also be returned to the agent

    def handle_output(self, code: str, comment: str):
        ...